In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import open3d as o3d
### for open3d important to have numpy<2.0.0 otherwise segmentation fault ###

import os
from pathlib import Path
import time
from tqdm import tqdm
from IPython.display import clear_output

from typing import Dict, Tuple, Any

from utils.reconstruction_metrics.depth_metrics import (
    absrel_depth_error,
    acc_under_threshold_depth_error,
    rms_depth_error,
    rms_log_depth_error,
)

from utils.reconstruction_metrics.pc_metrics import (
    chamfer_distance_p,
    chamfer_distance_l2_o3d,
    f_score_pcd,
)
from utils.data.data_worker import (
    load_scan,
    write_obj,
    write_obj_ply,
)
from utils.camera import (
    CameraModule
)
from utils.sfm import (
    SfmModule
)
from utils.data.data_processing import (
    reorganise_scene_folder,
    adjust_size_across_mask,
)
from utils.metric3d_mde import Metric3dMDE

np.set_printoptions(linewidth=200)

root_dir = './'
points_folder = os.path.join(root_dir, 'points')
scenes_folder = os.path.join(root_dir, 'posed_images')
get_api = lambda y: list(filter(lambda x: '__' not in x, dir(y)))
path_to_save = 'scene_reorganised'

In [ ]:
# python3 \
#     tr3d/tools/test.py \
#     tr3d/configs/tr3d/tr3d_scannet-3d-18class.py \
#     work_dirs/tr3d_scannet-3d-18class/latest.pth \
#     --eval \
#     mAP

# python3 \
#     tr3d/tools/test.py \
#     tr3d/configs/tr3d/tr3d_scannet-3d-18class.py \
#     tr3d_scannet.pth \
#     --eval \
#     mAP

## Предобработка сцены

In [3]:
### Реорганизация сцены ###
path_to_scene = os.path.join(scenes_folder, os.listdir(scenes_folder)[1])
path_to_save = 'scene_reorganised'
reorganise_scene_folder(path_to_scene, path_to_save)

In [4]:
path_to_scene

'./posed_images/scene0113_00'

In [5]:
### Подгон исходных изображений под размеры gt карт глубин ###
path_to_depth_maps = os.path.join(path_to_save, 'depth_maps')
path_to_images = os.path.join(path_to_save, 'images')
output_path = os.path.join(path_to_save, 'images_adjusted')
adjust_size_across_mask(
    path_to_mask=path_to_depth_maps,
    path_to_images=path_to_images,
    output_path=output_path,
)

In [6]:
### Анализ размеров изображений ###
path_to_images = os.path.join(path_to_save, 'images')
images = [
    np.array(Image.open(os.path.join(path_to_images, file)))
    for file in sorted(os.listdir(path_to_images))
]
images_unique_sizes = list(set([image.shape[:2] for image in images]))

path_to_images_adjusted = os.path.join(path_to_save, 'images_adjusted')
images_adjusted = [
    np.array(Image.open(os.path.join(path_to_images_adjusted, file)))
    for file in sorted(os.listdir(path_to_images_adjusted))
]
images_adjusted_unique_sizes = list(set([image.shape[:2] for image in images_adjusted]))

In [7]:
# Если на весь датасет только один размер картинок
# то обновляем параметры камеры
print('Unique sizes for original images', len(images_unique_sizes))
symbol = '-'
print(f'{symbol * 50}')
print('Unique sizes for adjusted images', len(images_adjusted_unique_sizes))

if len(images_adjusted_unique_sizes) == len(images_unique_sizes):
    path_to_camera = os.path.join(path_to_save, 'intrinsic.txt')
    camera = CameraModule(path_to_camera)
    camera_matrix = camera.get_camera_matrix()
    camera_matrix_adjusted = CameraModule.get_adapted_matrix(
        camera_matrix,
        images_adjusted_unique_sizes[0],
        images_unique_sizes[0]
    )
    camera_parameters_adjusted = CameraModule.convert_matrix_to_params(camera_matrix_adjusted)
    camera.update_camera_intrinsic(camera_parameters_adjusted)
    camera.save_camera_intrinsic(os.path.join(path_to_save, 'intrinsic_adjusted.txt'))

Unique sizes for original images 1
--------------------------------------------------
Unique sizes for adjusted images 1


## Predict Normals and Metric depth

In [5]:
crop_size = 5
metirc3d_model = Metric3dMDE(
    path_to_camera='scene_reorganised/intrinsic_adjusted.txt',
    crop_size=crop_size,
    model_type='vit',
)
images_processed, focal_scaled = \
    metirc3d_model.preprocess_images(path_to_images='scene_reorganised/images_adjusted')
name_to_depths = metirc3d_model.predict_processed(images_processed, focal_scaled)

Loading images:   0%|          | 0/300 [00:00<?, ?it/s]

Using cache found in /home/gribov/.cache/torch/hub/yvanyin_metric3d_main
/home/gribov/cv_work/rec_env/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/gribov/cv_work/rec_env/lib/python3.11/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/gribov/.cache/torch/hub/yvanyin_metric3d_main/mono/model/backbones/ViT_DINO.py:248: FutureWarning: xformers.components is deprecated and is not maintained anymore. It might be removed in a future version of xFormers 
  from xformers.components.attention import ScaledDotProduct


In [8]:
path_to_depths = os.path.join('scene_reorganised', 'depth_maps')

scene_depths = {
    file.split('.')[0]:np.array(Image.open(os.path.join(path_to_depths, file))) / 1000
    for file in sorted(os.listdir(path_to_depths))
}
all_images = sorted(list(scene_depths.keys()))

gts = np.stack([scene_depths[img][crop_size:-crop_size, crop_size:-crop_size] for img in all_images], axis=0)
preds = np.stack([name_to_depths[img] for img in all_images], axis=0)

In [ ]:
{
    'AbsRel': np.round(absrel_depth_error(gts, preds), 3),
    'Delta_1.25': np.round(acc_under_threshold_depth_error(gts, preds), 3),
    'RMS': np.round(rms_depth_error(gts, preds), 3),
    'RMS_log': np.round(rms_log_depth_error(gts, preds), 3),
}
# scene_folder[0]: {'AbsRel': 0.041, 'Delta_1.25': 0.994, 'RMS': 0.504, 'RMS_log': 0.059}
# scene_folder[1]: {'AbsRel': 0.051, 'Delta_1.25': 0.99, 'RMS': 0.652, 'RMS_log': 0.075}

{'AbsRel': 0.051, 'Delta_1.25': 0.99, 'RMS': 0.652, 'RMS_log': 0.075}

In [12]:
def read_bin(path):
    with open(path, "rb") as fid:
        width, height, channels = np.genfromtxt(
            fid, delimiter="&", max_rows=1, usecols=(0, 1, 2), dtype=int
        )
        fid.seek(0)
        num_delimiter = 0
        byte = fid.read(1)
        while True:
            if byte == b"&":
                num_delimiter += 1
                if num_delimiter >= 3:
                    break
            byte = fid.read(1)
        array = np.fromfile(fid, np.float32)
    array = array.reshape((width, height, channels), order="F")
    return np.transpose(array, (1, 0, 2)).squeeze()

colmap_depths = list()
path_to_colmap_depths = os.path.join(path_to_save, 'reconstruction', 'dense', 'stereo', 'depth_maps')
for file in sorted(os.listdir(path_to_colmap_depths)):
    name, way_to_get_depths = os.path.splitext(os.path.splitext(file)[0])
    way_to_get_depths = way_to_get_depths[1:]
    if way_to_get_depths == 'photometric':
        continue
    depth_map = read_bin(os.path.join(path_to_colmap_depths, file))
    colmap_depths.append((os.path.splitext(name)[0], depth_map))

colmap_depths = {n:d for n, d in colmap_depths}
path_to_colmap_images = os.path.join(path_to_save, 'reconstruction', 'dense', 'images')
colmap_images = {
    os.path.splitext(n)[0]:np.array(Image.open(os.path.join(path_to_colmap_images, n)))
    for n in sorted(os.listdir(path_to_colmap_images))
}

In [17]:
exclude_names = set()
for name in colmap_depths.keys():
    colmap_d = colmap_depths[name]
    metric_d = name_to_depths[name]
    colmap_h, colmap_w = colmap_d.shape
    metric_h, metric_w = metric_d.shape
    dif_h, dif_w = colmap_h - metric_h, colmap_w - metric_w
    unpad_h = dif_h // 2
    unpad_w = dif_w // 2
    if dif_h < 0 or dif_w < 0 or unpad_h == 0 or unpad_w == 0:
        exclude_names.add(name)
        continue
    if dif_h % 2 == 0 and dif_h > 0:
        colmap_d = colmap_d[unpad_h:-unpad_h, :]
    elif dif_h > 0:
        colmap_d = colmap_d[unpad_h+1:-unpad_h, :]
    if dif_w % 2 == 0 and dif_w > 0:
        colmap_d = colmap_d[:, unpad_w:-unpad_w]
    elif dif_w > 0:
        colmap_d = colmap_d[:, unpad_w+1:-unpad_w]
    colmap_depths[name] = colmap_d

colmap_images_to_take = set(list(colmap_depths.keys())).difference(exclude_names)

colmap_depths_s = np.hstack([d.flatten() for n, d in colmap_depths.items() if n in colmap_images_to_take])
metric_depths_s = np.hstack([d.flatten() for n, d in name_to_depths.items() if n in colmap_images_to_take])
mask = np.logical_and(colmap_depths_s > 0, metric_depths_s > 0)
colmap_depth_median, colmap_depth_avg = np.median(colmap_depths_s[mask]), np.mean(colmap_depths_s[mask])
metric3d_depth_median, metric3d_depth_avg = np.median(metric_depths_s[mask]), np.mean(metric_depths_s[mask])

median_depth_coef, avg_depth_coef = \
    metric3d_depth_median / colmap_depth_median, \
    metric3d_depth_avg / colmap_depth_avg

In [18]:
median_depth_coef

0.3964856

## 2D -> 3D проектирование

In [19]:
def load_pose(path_to_pose: str) -> Dict[str, Tuple[float, float]]:
    with open(path_to_pose, "r", encoding="utf-8") as fin:
        numbers =fin\
            .read()\
            .replace('\n', ' ')\
            .strip()\
            .split(' ')
        numbers = np.array(list(map(float, numbers))).reshape(4, -1)
    R, t = numbers[:3, :3], numbers[:3, 3]
    return R, t

def load_scene_poses(path_to_poses: str) -> Dict[str, Tuple[np.ndarray, np.ndarray]]:
    scene_poses = dict()
    for pose_file in sorted(os.listdir(path_to_poses)):
        path_to_pose = os.path.join(path_to_poses, pose_file)
        pose_name = pose_file.split('.', maxsplit=1)[0]
        scene_poses[pose_name] = load_pose(path_to_pose)
    return scene_poses

def pixels_to_spatial_coords(rgb, depth, K, R, t):
    # x = P * X
    # P = K * [I|0] * C^-1
    # C ~ 4x4 pose matrix
    h, w = rgb.shape[:2]
    x_coord_grid = np.arange(w)[None, :].repeat(h, axis=0)
    y_coord_grid = np.arange(h)[:, None].repeat(w, axis=1)
    xy_grid = np.dstack((x_coord_grid, y_coord_grid, np.ones((h, w))))
    xy_grid = xy_grid.reshape(-1, 3).T

    projection_to_cam_space = np.dot(np.linalg.inv(K), xy_grid)
    projection_to_cam_space *= depth.flatten()[None, :]
    
    # projection_to_world_space = np.dot(R, projection_to_cam_space) + t[:, None]
    # projection_to_world_space = projection_to_world_space.T.reshape(h, w, 3)
    
    R_cw = R.T
    t_cw = -np.dot(R_cw, t)
    projection_to_world_space = np.dot(R_cw, projection_to_cam_space) + t_cw[:, None]
    projection_to_world_space = projection_to_world_space.T.reshape(h, w, 3)

    return np.dstack((projection_to_world_space, rgb))

In [20]:
poses = load_scene_poses(os.path.join(path_to_save, 'poses'))
for key, val in poses.items():
    R, t = val
    poses[key] = np.vstack((np.hstack((R, t[:, None])), np.array([[0., 0., 0., 1.]])))

cam_module = CameraModule(os.path.join(path_to_save, 'intrinsic_adjusted.txt'))
cam_params = cam_module.get_camera_intrinsic()
fx, fy = cam_params['focal_p']
cx, cy = cam_params['optical_p']

In [29]:
volume = o3d.pipelines.integration.ScalableTSDFVolume(
    voxel_length=8.0 / 512.0,
    sdf_trunc=0.1,
    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
)

sfm = SfmModule(os.path.join(path_to_save, 'reconstruction', 'dense', 'sparse'))
# sfm.reconstruction.normalize()
path_to_images = os.path.join(path_to_save, 'images_adjusted')
name_to_image = {
    image.split('.')[0]:np.array(Image.open(os.path.join(path_to_images, image)))
    for image in sorted(os.listdir(path_to_images))
}
camera_to_k = sfm.get_cameras_info()
image_to_info = sfm.get_images_info()

all_images = sorted(list(image_to_info.keys()))
for i in tqdm(range(len(all_images)), leave=False):
    name = all_images[i]
    color, depth = \
        name_to_image[name], name_to_depths[name]
    # color, depth = \
    #     colmap_images[name], colmap_depths[name]
    # color, depth = \
    #     name_to_image[name], scene_depths[name].astype(np.float32)
    h, w = color.shape[:2]
    R, t = image_to_info[name]['rotation'], image_to_info[name]['translation']
    t *= median_depth_coef
    extrinsic = np.vstack((np.hstack((R, t[:, None])), np.array([[0.0, 0.0, 0.0, 1.0]])))
    # extrinsic = poses[name]
    if depth.shape != color.shape[:2]:
        depth = np.pad(depth, ((crop_size, crop_size), (crop_size, crop_size)), constant_values=0)
    color, depth = \
        o3d.cuda.pybind.geometry.Image(color), o3d.cuda.pybind.geometry.Image(depth)
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, depth_trunc=10.0, convert_rgb_to_intensity=False, depth_scale=1.0
    )
    cam_params = camera_to_k[image_to_info[name]['camera_id']]
    fx, fy = cam_params['focal_p']
    cx, cy = cam_params['optical_p']
    camera_o3d = o3d.camera.PinholeCameraIntrinsic(w, h, fx, fy, cx, cy)
    volume.integrate(
        rgbd, camera_o3d,
        extrinsic,
        # np.linalg.inv(extrinsic),
    )

In [31]:
pcd = volume.extract_point_cloud()
print(pcd)
# pcd = np.hstack((np.asarray(pcd.points), (np.asarray(pcd.colors) * 255).astype(np.uint8)))
# o3d.io.write_point_cloud('pred2.ply', pcd)

PointCloud with 480763 points.
[Open3D WARNING] Write Ply clamped color value to valid range


True

### Alignment

In [41]:
gt_scene = o3d.io.read_point_cloud('reconstructed_point_clouds/scene_gt2.ply')
pred_scene = o3d.io.read_point_cloud('reconstructed_point_clouds/sfm_metric3d.ply')

pred_box = pred_scene.get_oriented_bounding_box()
pred_scene.translate(-pred_box.center)
pred_box = pred_scene.get_oriented_bounding_box()
pred_scene.rotate(pred_box.R.T)

gt_box = gt_scene.get_oriented_bounding_box()
gt_scene.translate(-gt_box.center)

save_rough_align_pcd = False
if save_rough_align_pcd:
    print('saving')
    o3d.io.write_point_cloud("pred_scene_aligned.ply", pred_scene)
    o3d.io.write_point_cloud("gt_scene_aligned.ply", gt_scene)

downsampled = True
if downsampled:
    print('downsampling')
    voxel_size=0.1 # 8.0 / 512.0 # 0.015625
    radius = voxel_size
    pred_scene = pred_scene.voxel_down_sample(radius)
    gt_scene = gt_scene.voxel_down_sample(radius)
else:
    voxel_size = 8.0 / 512.0
    
pred_scene, gt_scene


downsampling


(PointCloud with 24090 points., PointCloud with 18888 points.)

In [42]:
print('fpfh')
radius_feature = voxel_size * 2
gt_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    gt_scene, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=90)
)
pred_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    pred_scene, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=90)
)

print('normals')
gt_scene.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
pred_scene.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))

print('ransac')
result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source=pred_scene,
    target=gt_scene,
    source_feature=gt_fpfh,
    target_feature=pred_fpfh,
    mutual_filter=True,
    max_correspondence_distance=voxel_size * 50,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n=20,
    checkers = [
        # o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(voxel_size * 100),
        # o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(np.pi)
    ],
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999),
)

fpfh
normals
ransac


In [43]:
print(np.round(result_ransac.transformation, 2))
pred_scene.transform(result_ransac.transformation)
# o3d.io.write_point_cloud('pred_scene_aligned.ply', pred_scene)
# o3d.io.write_point_cloud('gt_scene_aligned.ply', gt_scene)
np.save('ransac_transformation.npy', result_ransac.transformation)

[[ 0.24 -0.94 -0.24 -0.48]
 [ 0.95  0.28 -0.13  0.7 ]
 [ 0.19 -0.19  0.96 -0.79]
 [ 0.    0.    0.    1.  ]]


In [44]:
gt_scene = o3d.io.read_point_cloud('reconstructed_point_clouds/scene_gt2.ply')
pred_scene = o3d.io.read_point_cloud('reconstructed_point_clouds/sfm_metric3d.ply')
result_ransac = np.load('ransac_transformation.npy')

pred_box = pred_scene.get_oriented_bounding_box()
pred_scene.translate(-pred_box.center)
pred_box = pred_scene.get_oriented_bounding_box()
pred_scene.rotate(pred_box.R.T)
# transform according ransac icp
pred_scene.transform(result_ransac)

gt_box = gt_scene.get_oriented_bounding_box()
gt_scene.translate(-gt_box.center)

downsampled = False
if downsampled:
    print('downsampling')
    voxel_size=0.1 # 8.0 / 512.0 # 0.015625
    radius = voxel_size
    pred_scene = pred_scene.voxel_down_sample(radius)
    gt_scene = gt_scene.voxel_down_sample(radius)
else:
    voxel_size = 8.0 / 512.0
    
result_ransac, pred_scene, gt_scene, voxel_size


(array([[ 0.24043631, -0.94171712, -0.23528546, -0.482045  ],
        [ 0.9523481 ,  0.27573163, -0.13040389,  0.70039514],
        [ 0.18767922, -0.19271983,  0.9631384 , -0.79098249],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 PointCloud with 1356735 points.,
 PointCloud with 862025 points.,
 0.015625)

In [45]:
print('estimating normals')
pred_scene.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=50)
)
gt_scene.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=50)
)

print('icp')
result_icp = o3d.pipelines.registration.registration_colored_icp(
    pred_scene, gt_scene, voxel_size * 10, np.eye(4),
    o3d.pipelines.registration.TransformationEstimationForColoredICP(),
    o3d.pipelines.registration.ICPConvergenceCriteria(
        relative_fitness=1e-6,
        relative_rmse=1e-6,
        max_iteration=10000
    )
)

np.round(result_icp.transformation, 2), result_icp.inlier_rmse, result_icp.fitness
np.save('colored_icp_transformation.npy', result_icp.transformation)

estimating normals
icp


In [46]:
pred_scene.transform(result_icp.transformation)
o3d.io.write_point_cloud("gt_scene_aligned.ply", gt_scene)
o3d.io.write_point_cloud("pred_scene_aligned.ply", pred_scene)

True

### Metrics

In [38]:
gt_scene = o3d.io.read_point_cloud('gt2.ply')
pred_scene = o3d.io.read_point_cloud('pred2.ply')
result_ransac = np.load('ransac_transformation.npy')
result_icp = np.load('colored_icp_transformation.npy')

pred_box = pred_scene.get_oriented_bounding_box()
pred_scene.translate(-pred_box.center)
pred_box = pred_scene.get_oriented_bounding_box()
pred_scene.rotate(pred_box.R.T)
# transform according ransac icp
pred_scene.transform(result_ransac)
pred_scene.transform(result_icp)

gt_box = gt_scene.get_oriented_bounding_box()
gt_scene.translate(-gt_box.center)

PointCloud with 303663 points.

In [39]:
c_l1 = np.round(chamfer_distance_p(np.asarray(pred_scene.points), np.asarray(gt_scene.points)), 5)
c_l2 = np.round(chamfer_distance_l2_o3d(np.asarray(pred_scene.points), np.asarray(gt_scene.points)), 5)
f_score_1 = np.round(f_score_pcd(np.asarray(pred_scene.points), np.asarray(gt_scene.points), voxel_size * 1), 5)
f_score_2 = np.round(f_score_pcd(np.asarray(pred_scene.points), np.asarray(gt_scene.points), voxel_size * 2), 5)
f_score_3 = np.round(f_score_pcd(np.asarray(pred_scene.points), np.asarray(gt_scene.points), voxel_size * 3), 5)

In [40]:
c_l1, c_l2, f_score_1, f_score_2, f_score_3

(0.20644, 0.15114, 0.25313, 0.48242, 0.63633)